In [1]:
# Code to be able to import local modules in notebooks
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:

# Import dependencies
# Standard python libraries
import os

from oauthlib.oauth2 import WebApplicationClient
import requests
import pandas as pd

# Internal imports
from ffpackage.scraping import mfl
# from ffpackage.scraping import ffdb
# from ffpackage.scraping import ourlads
#from ffpackage.viz import viz

from appmanager.database import db
from appmanager.user import user


# Configuration (These variables are stored as environment variables)
GOOGLE_CLIENT_ID = os.environ.get("GOOGLE_CLIENT_ID", None)
GOOGLE_CLIENT_SECRET = os.environ.get("GOOGLE_CLIENT_SECRET", None)
GOOGLE_DISCOVERY_URL = ("https://accounts.google.com/.well-known/openid-configuration")

In [3]:
#def getFranchise():
user_league = "53906"

franchises = mfl.get_franchises(user_league)['franchiseName'].to_list()

franchises

['Providence Monastic Spirits',
 'Washington Fantasy Football Team',
 'Verdansk Vaqueros FFC',
 'Down with the Sickness unto Death',
 'Crocodilopolis Body Snatchers',
 "O'Henry's Stiffies",
 'Comeback Kings',
 'WHAM! BAM! Thank You CAM!',
 'Pretty Big Wieners',
 'Saskatoon Squatches',
 'Gus The Bus',
 'IDK much about soccer ']

In [4]:
#def allPlayers():
player_df = db.get_df("player_df")
tables = [player_df.to_html(classes='data')]
titles=player_df.columns.values
#return render_template("allPlayers.html", tables=tables, titles=titles)

/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [5]:
#@app.route('/waiverWire', methods=['GET', 'POST'])
#@login_required
#def waiverWire():
user_league = "53906"
user_franchise = "0005"

# Get Franchises in the league
franchise_df = mfl.get_franchises(user_league)
# Append a row to carry free agents
franchise_df = franchise_df.append({"franchiseID":"FA", "franchiseName":"Free Agent"}, ignore_index=True)

# Get franchise rosters
rosters_df = mfl.get_rosters(user_league, user_franchise=user_franchise)

# Get Free Agents
freeAgent_df = mfl.get_freeAgents(user_league)

# Combine Franchise rosters with free agents to get all players
rosters_df = pd.concat([rosters_df, freeAgent_df], axis=0)

# Get all players, sharkRank, and ADP
player_df = db.get_df("predictions")

# Merge all dfs
complete = player_df.merge(rosters_df, on='id_mfl', how='left').merge(franchise_df[['franchiseID', 'franchiseName']], on='franchiseID', how='left')
complete = complete[complete['franchiseID'].notna()]
complete = complete.sort_values(by=['pred'], ascending=False)
complete.reset_index(inplace=True, drop=True)
complete = complete[['player', 'age', 'team', 'franchiseName', 'pos', 'posRank', 'KR', 'PR', 'RES', 'pred', 'sharkAbsolute', 'adpAbsolute']]
complete.rename(columns={
    'player':'Player',
    'age':'Age',
    'team':'Team',
    'pos':'Position',
    'posRank': 'Rank',
    'pred': 'My Prediction',
    'sharkAbsolute': 'FantasySharks Prediction',
    'adpAbsolute': 'ADP-Based Prediction'
}, inplace=True)
complete.set_index('Player', drop=True, inplace=True)

return render_template("waiverWire.html", tables=[complete.to_html(classes='data')], titles=complete.columns.values)

/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_8250/793236653.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  franchise_df = franchise_df.append({"franchiseID":"FA", "franchiseName":"Free Agent"}, ignore_index=True)


,franchiseID,week,id_mfl,rosterStatus
0,0005,4,0507,ROSTER
1,0005,4,13403,ROSTER
2,0005,4,14783,ROSTER
3,0005,4,14073,ROSTER
4,0005,4,13672,ROSTER
...,...,...,...,...
748,FA,,14592,Free Agent
749,FA,,15696,Free Agent
750,FA,,10868,Free Agent
751,FA,,13585,Free Agent


In [6]:
#@app.route('/compareFranchises2')
#@login_required
#def compareFranchises2():
user_league = "53906"

# Get Franchises in the league
franchise_df = mfl.get_franchises(user_league)
franchise_df = franchise_df.append({"FranchiseID":"FA", "FranchiseName":"Free Agent"}, ignore_index=True)

# Get franchise rosters
rosters_df = mfl.get_rosters(user_league)

# Get Free Agents
freeAgent_df = mfl.get_freeAgents(user_league)

# Get all players, sharkRank, and ADP
predictions = db.get_df("predictions")

# Merge all dfs
complete = predictions.merge(rosters_df, on='id_mfl', how='left').merge(franchise_df[['franchiseID', 'franchiseName']], on='franchiseID', how='left')
complete['franchiseID'].fillna("FA", inplace=True)
complete['franchiseName'].fillna("Free Agent", inplace=True)
complete['rosterStatus'].fillna("Free Agent", inplace=True)
complete


/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_8250/2371207576.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  franchise_df = franchise_df.append({"FranchiseID":"FA", "FranchiseName":"Free Agent"}, ignore_index=True)
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


,id_mfl,player,age,team,pos,posRank,KR,PR,RES,sharkRank,adp,pred,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute,franchiseID,week,rosterStatus,franchiseName
0,5848,TOM BRADY,45.0,TBB,QB,QB1,NO,NO,NO,96.0,3000.00,347.798672,130.543335,288.143335,-129.969999,27.630001,0002,4,ROSTER,Washington Fantasy Football Team
1,14779,JUSTIN HERBERT,24.0,LAC,QB,QB1,NO,NO,NO,61.0,31.71,341.323461,171.646664,329.246664,171.646664,329.246664,0004,4,ROSTER,Down with the Sickness unto Death
2,13116,PATRICK MAHOMES,27.0,KCC,QB,QB1,NO,NO,NO,49.0,29.00,339.347453,178.046668,335.646667,178.046668,335.646667,0010,4,ROSTER,Saskatoon Squatches
3,9431,MATTHEW STAFFORD,34.0,LAR,QB,QB1,NO,NO,NO,116.0,3000.00,331.415001,118.080001,275.680001,-126.460000,31.140000,0007,4,ROSTER,Comeback Kings
4,7836,AARON RODGERS,38.0,GBP,QB,QB1,NO,NO,NO,80.0,3000.00,330.240378,153.929998,311.529998,-136.476666,21.123334,0009,4,ROSTER,Pretty Big Wieners
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,9921,ANDRE ROBERTS,34.0,CAR,WR,WR3,NO,NO,RES,1378.0,3000.00,25.630459,-97.993333,17.606666,-56.926666,58.673334,FA,NaN,Free Agent,Free Agent
649,13308,JAMAL AGNEW,27.0,JAC,WR,WR2,KR1,PR1,NO,903.0,3000.00,109.342865,-48.813333,150.786667,-110.340000,89.260000,0001,4,ROSTER,Providence Monastic Spirits
650,14838,LAVISKA SHENAULT,23.0,CAR,WR,WR3,NO,NO,NO,883.0,3000.00,24.121850,-47.473333,68.126667,-109.116667,6.483333,FA,NaN,Free Agent,Free Agent
651,14555,GUNNER OLSZEWSKI,25.0,PIT,WR,WR2,KR1,PR1,NO,1368.0,3000.00,107.820542,-94.883334,104.716666,-59.080001,140.519999,FA,NaN,Free Agent,Free Agent


In [ ]:

# Split complete df by player pos
qbs = complete[complete['pos'] == "QB"]
qbs.reset_index(inplace=True, drop=True)
rbs = complete[complete['pos'] == "RB"]
rbs.reset_index(inplace=True, drop=True)
wrs = complete[complete['pos'] == "WR"]
wrs.reset_index(inplace=True, drop=True)
tes = complete[complete['pos'] == "TE"]
tes.reset_index(inplace=True, drop=True)
pks = complete[complete['pos'] == "PK"]
pks.reset_index(inplace=True, drop=True)
defs = complete[complete['pos'] == "DF"]
defs.reset_index(inplace=True, drop=True)


### ADP Predictions
# Roster Builder logic
qbs_top = qbs.sort_values(by='adpAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(1)
rbs_top = rbs.sort_values(by='adpAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
wrs_top = wrs.sort_values(by='adpAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)
tes_top = tes.sort_values(by='adpAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
pks_top = pks.sort_values(by='adpAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
defs_top = defs.sort_values(by='adpAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)

qbs_remainder = qbs[~qbs['PlayerID'].isin(qbs_top['PlayerID'])].groupby('FranchiseName').head(1)
rbs_remainder = rbs[~rbs['PlayerID'].isin(rbs_top['PlayerID'])].groupby('FranchiseName').head(3)
wrs_remainder = wrs[~wrs['PlayerID'].isin(wrs_top['PlayerID'])].groupby('FranchiseName').head(3)
tes_remainder = tes[~tes['PlayerID'].isin(tes_top['PlayerID'])].groupby('FranchiseName').head(3)

remainder = pd.concat([qbs_remainder, rbs_remainder, wrs_remainder, tes_remainder])

top_remainders = remainder.sort_values(by='adpAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)

players_onthefield = pd.concat([qbs_top, rbs_top, wrs_top, tes_top, pks_top, defs_top, top_remainders])
players_onthefield = players_onthefield.sort_values(by='adpAbsolute', ascending=False, ignore_index=True)

fran_rank = players_onthefield.groupby('FranchiseName').sum().sort_values(by='adpAbsolute', ascending=False)

sorter = fran_rank.index

players_onthefield.FranchiseName = players_onthefield.FranchiseName.astype("category")
players_onthefield.FranchiseName.cat.set_categories(sorter, inplace=True)
players_onthefield.sort_values(["FranchiseName"], inplace=True)

# remove Free Agents
players_onthefield = players_onthefield.loc[players_onthefield.FranchiseID!="FA"]

# Find the lowest scoring player on the field and set them as the low bar
for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
    players_onthefield.loc[players_onthefield['pos']==x, 'adpComp'] = players_onthefield.loc[players_onthefield['pos']==x, 'adpAbsolute'].min()
players_onthefield['adpRelative'] = players_onthefield['adpAbsolute'] - players_onthefield['adpComp']

# Create bar chart
figADP = px.bar(players_onthefield, 
            x="FranchiseName", 
            y="adpRelative", 
            color="pos", 
            text='player', 
            color_discrete_map={
                "QB": "hsla(210, 60%, 25%, 1)", #blue #1033a6 #0c2987 1033a6 062647 #293745 
                "RB": "hsla(12, 50%, 45%, 1)", #gold #f5d000 ffa524 a23419 a34e39
                "WR": "hsla(267, 40%, 45%, 1)", #purple #4f22bc #643fc1 643fc1 621B74 675280
                "TE": "hsla(177, 68%, 36%, 1)", #teal #02687b #038097 1295ad 43B3AE
                "PK": "hsla(14, 30%, 40%, 1)", #gold #f5d000 ffa524 664e47
                "DF": "hsla(35, 70%, 65%, 1)"}, #gold #f5d000 ffa524 a49375 ffb54d
            category_orders={
                "pos": ["QB", "RB", "WR", "TE", "PK", "DF"]},
            hover_name="player",
            hover_data={
                'adpRelative':True, 'pred':True, 'sharkAbsolute':True, 'adpAbsolute':True,
                'player':False, 'pos':False, 'FranchiseName':False
                },
            labels={
                "FranchiseName":"Franchise",
                "adpRelative":"Player Value",
                "pred":"ChopBlock Prediction",
                "sharkAbsolute":"FantasySharks Prediction",
                "adpAbsolute":"ADP-Based Prediction"
            }
            )
figADP.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="ADP-Based Predictions",
            font_family="Skia",
            showlegend=False
            )

graphJSON_adp = json.dumps(figADP, cls=plotly.utils.PlotlyJSONEncoder)


### Shark Predictions
# Roster Builder logic
qbs_top = qbs.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(1)
rbs_top = rbs.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
wrs_top = wrs.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)
tes_top = tes.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
pks_top = pks.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
defs_top = defs.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)

qbs_remainder = qbs[~qbs['PlayerID'].isin(qbs_top['PlayerID'])].groupby('FranchiseName').head(1)
rbs_remainder = rbs[~rbs['PlayerID'].isin(rbs_top['PlayerID'])].groupby('FranchiseName').head(3)
wrs_remainder = wrs[~wrs['PlayerID'].isin(wrs_top['PlayerID'])].groupby('FranchiseName').head(3)
tes_remainder = tes[~tes['PlayerID'].isin(tes_top['PlayerID'])].groupby('FranchiseName').head(3)

remainder = pd.concat([qbs_remainder, rbs_remainder, wrs_remainder, tes_remainder])

top_remainders = remainder.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)

players_onthefield = pd.concat([qbs_top, rbs_top, wrs_top, tes_top, pks_top, defs_top, top_remainders])
players_onthefield = players_onthefield.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True)

fran_rank = players_onthefield.groupby('FranchiseName').sum().sort_values(by='sharkAbsolute', ascending=False)

sorter = fran_rank.index

players_onthefield.FranchiseName = players_onthefield.FranchiseName.astype("category")
players_onthefield.FranchiseName.cat.set_categories(sorter, inplace=True)
players_onthefield.sort_values(["FranchiseName"], inplace=True)

# remove Free Agents
players_onthefield = players_onthefield.loc[players_onthefield.FranchiseID!="FA"]

# Find the lowest scoring player on the field and set them as the low bar
for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
    players_onthefield.loc[players_onthefield['pos']==x, 'sharkComp'] = players_onthefield.loc[players_onthefield['pos']==x, 'sharkAbsolute'].min()
players_onthefield['sharkRelative'] = players_onthefield['sharkAbsolute'] - players_onthefield['sharkComp']

# Create bar chart
figShark = px.bar(players_onthefield, 
            x="FranchiseName", 
            y="sharkRelative", 
            color="pos", 
            text='player', 
            color_discrete_map={
                "QB": "hsla(210, 60%, 25%, 1)", #blue #1033a6 #0c2987 1033a6 062647 #293745 
                "RB": "hsla(12, 50%, 45%, 1)", #gold #f5d000 ffa524 a23419 a34e39
                "WR": "hsla(267, 40%, 45%, 1)", #purple #4f22bc #643fc1 643fc1 621B74 675280
                "TE": "hsla(177, 68%, 36%, 1)", #teal #02687b #038097 1295ad 43B3AE
                "PK": "hsla(14, 30%, 40%, 1)", #gold #f5d000 ffa524 664e47
                "DF": "hsla(35, 70%, 65%, 1)"}, #gold #f5d000 ffa524 a49375 ffb54d
            category_orders={
                "pos": ["QB", "RB", "WR", "TE", "PK", "DF"]},
            hover_name="player",
            hover_data={
                'sharkRelative':True, 'pred':True, 'sharkAbsolute':True, 'adpAbsolute':True,
                'player':False, 'pos':False, 'FranchiseName':False
                },
            labels={
                "FranchiseName":"Franchise",
                "sharkRelative":"Player Value",
                "pred":"ChopBlock Prediction",
                "sharkAbsolute":"FantasySharks Prediction",
                "adpAbsolute":"ADP-Based Prediction"
            }
            )
figShark.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="FantasySharks Predictions",
            font_family="Skia",
            showlegend=False
            )

graphJSON_shark = json.dumps(figShark, cls=plotly.utils.PlotlyJSONEncoder)



### My predictions
# Roster Builder logic
qbs_top = qbs.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(1)
rbs_top = rbs.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
wrs_top = wrs.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)
tes_top = tes.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
pks_top = pks.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
defs_top = defs.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)

qbs_remainder = qbs[~qbs['PlayerID'].isin(qbs_top['PlayerID'])].groupby('FranchiseName').head(1)
rbs_remainder = rbs[~rbs['PlayerID'].isin(rbs_top['PlayerID'])].groupby('FranchiseName').head(3)
wrs_remainder = wrs[~wrs['PlayerID'].isin(wrs_top['PlayerID'])].groupby('FranchiseName').head(3)
tes_remainder = tes[~tes['PlayerID'].isin(tes_top['PlayerID'])].groupby('FranchiseName').head(3)

remainder = pd.concat([qbs_remainder, rbs_remainder, wrs_remainder, tes_remainder])

top_remainders = remainder.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)

players_onthefield = pd.concat([qbs_top, rbs_top, wrs_top, tes_top, pks_top, defs_top, top_remainders])
players_onthefield = players_onthefield.sort_values(by='pred', ascending=False, ignore_index=True)

fran_rank = players_onthefield.groupby('FranchiseName').sum().sort_values(by='pred', ascending=False)

sorter = fran_rank.index

players_onthefield.FranchiseName = players_onthefield.FranchiseName.astype("category")
players_onthefield.FranchiseName.cat.set_categories(sorter, inplace=True)
players_onthefield.sort_values(["FranchiseName"], inplace=True)

# remove Free Agents
players_onthefield = players_onthefield.loc[players_onthefield.FranchiseID!="FA"]

# Find the lowest scoring player on the field and set them as the low bar
for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
    players_onthefield.loc[players_onthefield['pos']==x, 'predComp'] = players_onthefield.loc[players_onthefield['pos']==x, 'pred'].min()
players_onthefield['predRelative'] = players_onthefield['pred'] - players_onthefield['predComp']

# Create bar chart
figPred = px.bar(players_onthefield, 
            x="FranchiseName", 
            y="predRelative", 
            color="pos", 
            text='player', 
            color_discrete_map={
                "QB": "hsla(210, 60%, 25%, 1)", #blue #1033a6 #0c2987 1033a6 062647 #293745 
                "RB": "hsla(12, 50%, 45%, 1)", #gold #f5d000 ffa524 a23419 a34e39
                "WR": "hsla(267, 40%, 45%, 1)", #purple #4f22bc #643fc1 643fc1 621B74 675280
                "TE": "hsla(177, 68%, 36%, 1)", #teal #02687b #038097 1295ad 43B3AE
                "PK": "hsla(14, 30%, 40%, 1)", #gold #f5d000 ffa524 664e47
                "DF": "hsla(35, 70%, 65%, 1)"}, #gold #f5d000 ffa524 a49375 ffb54d
            category_orders={
                "pos": ["QB", "RB", "WR", "TE", "PK", "DF"]},
            hover_name="player",
            hover_data={
                'predRelative':True, 'pred':True, 'sharkAbsolute':True, 'adpAbsolute':True,
                'player':False, 'pos':False, 'FranchiseName':False
                },
            labels={
                "FranchiseName":"Franchise",
                "predRelative":"Player Value",
                "pred":"ChopBlock Prediction",
                "sharkAbsolute":"FantasySharks Prediction",
                "adpAbsolute":"ADP-Based Prediction"
            }
            )
figPred.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="ChopBlock Predictions",
            font_family="Skia",
            showlegend=False
            )

graphJSON_pred = json.dumps(figPred, cls=plotly.utils.PlotlyJSONEncoder)
#return render_template('compareFranchises2.html', graphJSON_pred=graphJSON_pred, graphJSON_adp=graphJSON_adp, graphJSON_shark=graphJSON_shark)
